In [1]:
import sys
sys.path.insert(0,'/home/lshtanko/Programming/another-fem-framework/')

In [2]:
#import h5py

from operator_assembler.assembly_interface import AssemblyInterface2D
from operator_assembler.n_to_1_ddof_allocator import Nto1Allocator2D
from grid.grid_manager import GridManager
from grid.allocator_interface import InterfaceDofAllocator2D
from grid.grid_domain import GridDomain
from operator_assembler.matrix_assembler import MatrixAssembler2D
from scipy.ndimage import imread
from grid import cell, grid_layer, grid_manager
import h5py
import numpy as np

from common import visual
from matplotlib import pyplot as plt
%matplotlib inline

#### Bad cell

In [ ]:
data_start = (16, 20)
data_shape = (4, 4)
data_end = tuple([i+j for i,j in zip(data_start, data_shape)])

h5_data = h5py.File('../../../GM_L3D/600/state/viz_GM_1.h5')['index'][0]
data = h5_data[data_start[0]:data_end[0], data_start[1]:data_end[1]]
#data = imread('/Users/marusy/Programming/model/bhi2_labelled0000.tif')[data_start[0]:data_end[0], data_start[1]:data_end[1]]

grid_domain = GridDomain(integer_size=data_shape, domain_size=(1.,1.))
gm = GridManager()
ifma = InterfaceDofAllocator2D(grid_manager=gm.fit(data=data))

gm.draw_grid()

In [3]:
cell1 = cell.Cell2D(size=(1,1), ll_vertex=(0,0))
cell2 = cell.Cell2D(size=(1,1), ll_vertex=(0,1))

cell3 = cell.Cell2D(size=(2,2), ll_vertex=(1,0))

In [4]:
layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2]], layer_number=0, index=[1,1], div_index=[1,1])
layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3]], layer_number=1, index=[1], div_index=[1])

In [5]:
gm = grid_manager.GridManager(dim=2, method='coarse2to1')
gm.grid_layers = [layer1, layer2]
ifma = InterfaceDofAllocator2D(grid_manager=gm)

In [6]:
ma = Nto1Allocator2D(grid_interface=ifma)
ma._make_ddof_index()
grid_domain = GridDomain(domain_size=(3.,2.), integer_size=(3,2))
grid_domain.make_pointwise_index(ma)

In [7]:
ifma2 = AssemblyInterface2D(allocator=ma)
MA = MatrixAssembler2D(assembly_interface=ifma2, grid_domain=grid_domain)
deep = MA.assemble()

{0, 1, 2, 4, 5, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 25, 26, 27} {32, 33, 34, 35, 36, 40, 41, 42, 43, 28, 29, 31} [[37, 38, 39, 30]] [[9, 10, 11, 3, 22, 23, 21]]
 0set() {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15} [] []
 1set() {0, 1, 2, 3, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27} [] []
 2

In [8]:
ma.get_cell_list_of_ddofs_global(cell=cell3)

[37, 38, 39, 30, 35, 40, 41, 28, 36, 42, 43, 29, 34, 32, 33, 31]

In [ ]:
sine_test = grid_domain.vectorize_function(lambda x,y: np.sin(x+y))
product = grid_domain.devectorize_vector(MA.assembled * sine_test)
init = grid_domain.devectorize_function(lambda x,y: np.sin(x+y))

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_vector(MA.assembled * sine_test),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_function(lambda x,y: np.sin(x+y)),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

In [9]:
from common.tests import *

In [13]:
partial_diff_test(matrix=MA.assembled.toarray(), point=grid_domain.normal_index[33], pointnum=33, pointdict=grid_domain.pointwise_index)

X derivative: -0.972222093877487
Y derivative: -0.0027161464934136147
XX derivative: -0.2777776915526216
YY derivative: -0.1600254319176028
XY_derivative: -0.005432227182431214



In [ ]:
visual.plot_sparse_pattern(matrix=MA.assembled)

In [ ]:
plt.imshow(deep[0][1].toarray(), interpolation='none')

In [ ]:
deep[0][0][0][21].toarray()

In [ ]:
deep[0][0][0][21].nonzero()

In [14]:
grid_domain.normal_index

{0: (0.27639320492744446, 1.0),
 1: (0.72360682487487793, 1.0),
 2: (0.0, 1.0),
 3: (1.0, 1.0),
 4: (0.0, 0.27639320492744446),
 5: (0.0, 0.72360682487487793),
 6: (0.0, 0.0),
 7: (0.27639320492744446, 0.0),
 8: (0.72360682487487793, 0.0),
 9: (1.0, 0.0),
 10: (1.0, 0.27639320492744446),
 11: (1.0, 0.72360682487487793),
 12: (0.27639320492744446, 0.27639320492744446),
 13: (0.27639320492744446, 0.72360682487487793),
 14: (0.72360682487487793, 0.27639320492744446),
 15: (0.72360682487487793, 0.72360682487487793),
 16: (0.0, 1.2763931751251221),
 17: (0.0, 1.7236068248748779),
 18: (0.0, 2.0),
 19: (0.27639320492744446, 2.0),
 20: (0.72360682487487793, 2.0),
 21: (1.0, 2.0),
 22: (1.0, 1.2763931751251221),
 23: (1.0, 1.7236068248748779),
 24: (0.27639320492744446, 1.2763931751251221),
 25: (0.27639320492744446, 1.7236068248748779),
 26: (0.72360682487487793, 1.2763931751251221),
 27: (0.72360682487487793, 1.7236068248748779),
 28: (1.5527863502502441, 2.0),
 29: (2.4472136497497559, 2.0)